# Ensemble Learning

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Load Dataset

In [2]:
!gdown 1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3 -O data/Housing.csv

Downloading...
From: https://drive.google.com/uc?id=1qeJqFtRdjjHqExbWJcgKy0yJbczTTAE3
To: d:\AIO2024\study_project\AIO2024_Exercises\module03\week04_ensemble_learning\data\Housing.csv

  0%|          | 0.00/30.0k [00:00<?, ?B/s]
100%|██████████| 30.0k/30.0k [00:00<00:00, 894kB/s]


In [3]:
dataset_path = "./data/Housing.csv"
df = pd.read_csv(dataset_path)
df.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


## Encoding Categorical Features and Combining with Numerical Data

In [5]:
categorical_cols = df.select_dtypes(include=["object"]).columns.to_list()
categorical_cols

['mainroad',
 'guestroom',
 'basement',
 'hotwaterheating',
 'airconditioning',
 'prefarea',
 'furnishingstatus']

In [8]:
# Encode categorical columns using OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
encoded_categorical_cols = ordinal_encoder.fit_transform(df[categorical_cols])

encoded_categorical_df = pd.DataFrame(encoded_categorical_cols, columns=categorical_cols)

# Drop original categorical columns, keeping only numerical data
numerical_df = df.drop(categorical_cols, axis=1)

# Combine numerical data with encoded categorical columns
encoded_df = pd.concat([numerical_df, encoded_categorical_df], axis=1)
encoded_df.head()

,price,area,bedrooms,bathrooms,stories,parking,mainroad,guestroom,basement,hotwaterheating,airconditioning,prefarea,furnishingstatus
0,13300000,7420,4,2,3,2,1.0,0.0,0.0,0.0,1.0,1.0,0.0
1,12250000,8960,4,4,4,3,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,12250000,9960,3,2,2,2,1.0,0.0,1.0,0.0,0.0,1.0,1.0
3,12215000,7500,4,2,2,3,1.0,0.0,1.0,0.0,1.0,1.0,0.0
4,11410000,7420,4,1,2,2,1.0,1.0,1.0,0.0,1.0,0.0,0.0


## Standardizing the Dataset Using StandardScaler

In [12]:
normalizer = StandardScaler()
dataset_arr = normalizer.fit_transform(encoded_df)
dataset_arr[:2]

array([[ 4.56636513,  1.04672629,  1.40341936,  1.42181174,  1.37821692,
         1.51769249,  0.40562287, -0.46531479, -0.73453933, -0.2192645 ,
         1.4726183 ,  1.80494113, -1.40628573],
       [ 4.00448405,  1.75700953,  1.40341936,  5.40580863,  2.53202371,
         2.67940935,  0.40562287, -0.46531479, -0.73453933, -0.2192645 ,
         1.4726183 , -0.55403469, -1.40628573]])

## Splitting the Dataset into Training and Validation Sets

In [15]:
X, y = dataset_arr[:, 1:], dataset_arr[:, 0]

X.shape, y.shape

((545, 12), (545,))

In [16]:
test_size = 0.3
random_state = 1
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size = test_size,
    random_state = random_state,
    shuffle=is_shuffle, 
)

## Training a Evaluating

### Random Forest Regressor

In [18]:
rf_regressor = RandomForestRegressor(random_state=random_state)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(random_state=1)

In [32]:
y_pred_rf = rf_regressor.predict(X_val)

mae = mean_absolute_error(y_val, y_pred_rf)
mse = mean_squared_error(y_val, y_pred_rf)

print("Random Forest - Evaluations results on validation set: ")
print("Mean Absolute Error: ", mae)
print("Mean Squared Error: ", mse)

Random Forest - Evaluations results on validation set: 
Mean Absolute Error:  0.46093873321571177
Mean Squared Error:  0.37944418523089524


### AdaBoost Regressor

In [22]:
ab_regressor = AdaBoostRegressor(random_state=random_state)
ab_regressor.fit(X_train, y_train)

AdaBoostRegressor(random_state=1)

In [29]:
y_pred_ab = ab_regressor.predict(X_val)

mae = mean_absolute_error(y_val, y_pred_ab)
mse = mean_squared_error(y_val, y_pred_ab)

print("AdaBoost - Evaluations results on validation set: ")
print("Mean Absolute Error: ", mae)
print("Mean Squared Error: ", mse)

AdaBoost - Evaluations results on validation set: 
Mean Absolute Error:  0.567680019897059
Mean Squared Error:  0.5739244030038942
